In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets
!pip install peft
!pip install evaluate scikit-learn transformers[torch]
!pip install accelerate>=0.26.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/cu121
ERROR: Could not find a version that satisfies the requirement torchaudio (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for torchaudio


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


zsh:1: 0.26.0 not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Prepare the dataset

In [41]:
from datasets import load_dataset

imdb = load_dataset("stanfordnlp/imdb")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = imdb.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [42]:
training_set = tokenized_datasets['train'].shuffle(42).select(range(5000))
shuffled_set =  tokenized_datasets['test'].shuffle(42)
validation_set  = shuffled_set.select(range(1000))
test_set = shuffled_set.select(range(1000, 2000))

## First method : Using the transformers library and pytorch


In [43]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification



## Fine-tuning the whole model

In [44]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## LoRA implementation

In [45]:
[k for k,v  in model.bert.named_parameters()]

['embeddings.word_embeddings.weight',
 'embeddings.position_embeddings.weight',
 'embeddings.token_type_embeddings.weight',
 'embeddings.LayerNorm.weight',
 'embeddings.LayerNorm.bias',
 'encoder.layer.0.attention.self.query.weight',
 'encoder.layer.0.attention.self.query.bias',
 'encoder.layer.0.attention.self.key.weight',
 'encoder.layer.0.attention.self.key.bias',
 'encoder.layer.0.attention.self.value.weight',
 'encoder.layer.0.attention.self.value.bias',
 'encoder.layer.0.attention.output.dense.weight',
 'encoder.layer.0.attention.output.dense.bias',
 'encoder.layer.0.attention.output.LayerNorm.weight',
 'encoder.layer.0.attention.output.LayerNorm.bias',
 'encoder.layer.0.intermediate.dense.weight',
 'encoder.layer.0.intermediate.dense.bias',
 'encoder.layer.0.output.dense.weight',
 'encoder.layer.0.output.dense.bias',
 'encoder.layer.0.output.LayerNorm.weight',
 'encoder.layer.0.output.LayerNorm.bias',
 'encoder.layer.1.attention.self.query.weight',
 'encoder.layer.1.attention.se

In [46]:
print(f"What is the model \n\n {model}")


What is the model 

 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

### Create the LoRA module

We will consider in this module the original linear, and the down and up projection

In [47]:
import torch
from torch import nn

class LoRALinear(nn.Module):
    def __init__(self, in_dim: int, out_dim: int, rank: int):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim, bias=True)
        self.lora_a = nn.Linear(in_dim, rank, bias=False)
        self.lora_b = nn.Linear(rank, out_dim, bias=False)

        nn.init.zeros_(self.lora_a.weight)
        nn.init.zeros_(self.lora_b.weight)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x) + self.lora_b(self.lora_a(x))


### Create a function replacing linear by lora module
From the original Linear module return a LoRA module
* The linear of the LoRA linear must be initialised with pretrained weights

In [48]:
def linear_to_lora(linear):
    weight = linear.weight.data
    bias = linear.bias.data if linear.bias is not None else None

    out_dim, in_dim = weight.shape
    lora = LoRALinear(in_dim, out_dim, rank=8)

    lora.linear.weight.data = weight.clone()
    if bias is not None:
        lora.linear.bias.data = bias.clone()

    return lora


We now replace the target linear by the LoRALinear described above

In [49]:
import copy

lora_model = copy.deepcopy(model)
lora_parameters = []

for block in lora_model.bert.encoder.layer:
    block.attention.self.query = linear_to_lora(block.attention.self.query)
    block.attention.self.key   = linear_to_lora(block.attention.self.key)
    block.attention.self.value = linear_to_lora(block.attention.self.value)

    lora_parameters += list(block.attention.self.query.lora_a.parameters())
    lora_parameters += list(block.attention.self.query.lora_b.parameters())
    lora_parameters += list(block.attention.self.key.lora_a.parameters())
    lora_parameters += list(block.attention.self.key.lora_b.parameters())
    lora_parameters += list(block.attention.self.value.lora_a.parameters())
    lora_parameters += list(block.attention.self.value.lora_b.parameters())


In [50]:
print(f"What is the model modified with LoRA: \n\n {lora_model}")


What is the model modified with LoRA: 

 BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): LoRALinear(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora_a): Linear(in_features=768, out_features=8, bias=False)
                (lora_b): Linear(in_features=8, out_features=768, bias=False)
              )
              (key): LoRALinear(
                (linear): Linear(in_features=768, out_features=768, bias=True)
                (lora

### Defining module requiring grad

In [51]:
for k,v  in lora_model.bert.named_parameters():
    print(k)
    if ('lora' in k):
        v.requires_grad = True
    else:
        v.requires_grad = False

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.linear.weight
encoder.layer.0.attention.self.query.linear.bias
encoder.layer.0.attention.self.query.lora_a.weight
encoder.layer.0.attention.self.query.lora_b.weight
encoder.layer.0.attention.self.key.linear.weight
encoder.layer.0.attention.self.key.linear.bias
encoder.layer.0.attention.self.key.lora_a.weight
encoder.layer.0.attention.self.key.lora_b.weight
encoder.layer.0.attention.self.value.linear.weight
encoder.layer.0.attention.self.value.linear.bias
encoder.layer.0.attention.self.value.lora_a.weight
encoder.layer.0.attention.self.value.lora_b.weight
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encode

## Using transformers Trainer to fine-tune with LoRA 

In [52]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [53]:
import accelerate.utils.memory

if not hasattr(accelerate.utils.memory, "clear_device_cache"):
    def clear_device_cache():
        pass
    accelerate.utils.memory.clear_device_cache = clear_device_cache


In [54]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_custom_lora", 
                                  eval_strategy="steps",
                                  eval_steps= 128,
                                  num_train_epochs=2,)
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset= training_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)


In [55]:
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.6796972155570984, 'eval_accuracy': 0.596, 'eval_runtime': 29.1831, 'eval_samples_per_second': 34.266, 'eval_steps_per_second': 4.283, 'epoch': 0.2}


/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


...

{'eval_loss': 0.6577138900756836, 'eval_accuracy': 0.631, 'eval_runtime': 28.6689, 'eval_samples_per_second': 34.881, 'eval_steps_per_second': 4.36, 'epoch': 1.64}


/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.6574687361717224, 'eval_accuracy': 0.633, 'eval_runtime': 28.7038, 'eval_samples_per_second': 34.839, 'eval_steps_per_second': 4.355, 'epoch': 1.84}
{'train_runtime': 1011.0103, 'train_samples_per_second': 9.891, 'train_steps_per_second': 1.236, 'train_loss': 0.6796511352539063, 'epoch': 2.0}


TrainOutput(global_step=1250, training_loss=0.6796511352539063, metrics={'train_runtime': 1011.0103, 'train_samples_per_second': 9.891, 'train_steps_per_second': 1.236, 'total_flos': 2644700098560000.0, 'train_loss': 0.6796511352539063, 'epoch': 2.0})

In [ ]:
import pandas as pd
import math

logs = trainer.state.log_history

train_losses = [
    (log["step"], log["loss"])
    for log in logs
    if "loss" in log and "eval_loss" not in log
]

def get_closest_train_loss(step):
    candidates = [loss for s, loss in train_losses if s <= step]
    if len(candidates) == 0:
        return math.nan
    return candidates[-1]

rows = []
for log in logs:
    if "eval_loss" in log:
        step = log["step"]
        rows.append({
            "Step": step,
            "Training Loss": round(get_closest_train_loss(step), 4),
            "Validation Loss": round(log["eval_loss"], 4),
            "Accuracy": round(log["eval_accuracy"], 4)
        })

df_results = pd.DataFrame(rows)
df_results


,Step,Training Loss,Validation Loss,Accuracy
0,128,NaN,0.6797,0.596
1,256,NaN,0.6732,0.618
2,384,NaN,0.6690,0.638
3,512,0.6846,0.6641,0.634
4,640,0.6846,0.6702,0.587
5,768,0.6846,0.6599,0.627
6,896,0.6846,0.6599,0.619
7,1024,0.6750,0.6577,0.631
8,1152,0.6750,0.6575,0.633


## Using PEFT library to fine-tune with LoRA 

In [62]:
from peft import LoraConfig, TaskType,  get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    num_labels=2
)
peft_model = get_peft_model(model, lora_config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [63]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_custom_lora",
    evaluation_strategy="steps",
    eval_steps=128,
    logging_strategy="steps",
    logging_steps=128,
    num_train_epochs=2,
    report_to="none",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset= training_set,
    eval_dataset=validation_set,
    compute_metrics=compute_metrics,
)

/opt/anaconda3/lib/python3.13/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [64]:
trainer.train()

  0%|          | 0/1250 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.7068, 'grad_norm': 3.667567729949951, 'learning_rate': 4.488e-05, 'epoch': 0.2}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.6885630488395691, 'eval_accuracy': 0.551, 'eval_runtime': 28.8171, 'eval_samples_per_second': 34.702, 'eval_steps_per_second': 4.338, 'epoch': 0.2}
{'loss': 0.6895, 'grad_norm': 3.7987382411956787, 'learning_rate': 3.9760000000000006e-05, 'epoch': 0.41}


...

{'eval_loss': 0.621903657913208, 'eval_accuracy': 0.653, 'eval_runtime': 28.6308, 'eval_samples_per_second': 34.927, 'eval_steps_per_second': 4.366, 'epoch': 1.64}
{'loss': 0.6332, 'grad_norm': 9.44797420501709, 'learning_rate': 3.92e-06, 'epoch': 1.84}


/opt/anaconda3/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.6183028817176819, 'eval_accuracy': 0.668, 'eval_runtime': 28.8974, 'eval_samples_per_second': 34.605, 'eval_steps_per_second': 4.326, 'epoch': 1.84}
{'train_runtime': 1013.908, 'train_samples_per_second': 9.863, 'train_steps_per_second': 1.233, 'train_loss': 0.6692172241210937, 'epoch': 2.0}


TrainOutput(global_step=1250, training_loss=0.6692172241210937, metrics={'train_runtime': 1013.908, 'train_samples_per_second': 9.863, 'train_steps_per_second': 1.233, 'total_flos': 2632290263040000.0, 'train_loss': 0.6692172241210937, 'epoch': 2.0})

In [65]:
logs = trainer.state.log_history

train_loss_by_step = {}
eval_by_step = {}

for log in logs:
    if "loss" in log and "eval_loss" not in log:
        train_loss_by_step[log["step"]] = log["loss"]
    if "eval_loss" in log:
        eval_by_step[log["step"]] = {
            "Validation Loss": log["eval_loss"],
            "Accuracy": log.get("eval_accuracy")
        }

rows = []
for step in sorted(eval_by_step.keys()):
    rows.append({
        "Step": step,
        "Training Loss": round(train_loss_by_step.get(step, float("nan")), 4),
        "Validation Loss": round(eval_by_step[step]["Validation Loss"], 4),
        "Accuracy": round(eval_by_step[step]["Accuracy"], 4),
    })

df_peft_results = pd.DataFrame(rows)
df_peft_results


,Step,Training Loss,Validation Loss,Accuracy
0,128,0.7068,0.6886,0.551
1,256,0.6895,0.6797,0.595
2,384,0.6873,0.6704,0.624
3,512,0.6871,0.6594,0.635
4,640,0.6762,0.6598,0.610
5,768,0.6590,0.6365,0.652
6,896,0.6475,0.6270,0.656
7,1024,0.6478,0.6219,0.653
8,1152,0.6332,0.6183,0.668


## Compare the results of the different approaches

In [66]:
comparison_df = pd.DataFrame({
    "Approach": ["Custom LoRA", "PEFT LoRA"],
    "Final Training Loss": [
        df_results["Training Loss"].dropna().iloc[-1],
        df_peft_results["Training Loss"].iloc[-1]
    ],
    "Final Validation Loss": [
        df_results["Validation Loss"].iloc[-1],
        df_peft_results["Validation Loss"].iloc[-1]
    ],
    "Final Accuracy": [
        df_results["Accuracy"].iloc[-1],
        df_peft_results["Accuracy"].iloc[-1]
    ]
})

comparison_df


,Approach,Final Training Loss,Final Validation Loss,Final Accuracy
0,Custom LoRA,0.6750,0.6575,0.633
1,PEFT LoRA,0.6332,0.6183,0.668
